In [1]:
import numpy as np
import networkit
import networkx as nx

DATA_FILENAME = '/home/sami/py-graph/data/oneshot_fennel_weights.txt'

# Reading data
# - neither networkit nor networkx handle node weights
# - networkit can read the METIS file format, networkx can't
# - networkit does not support extra attributes to nodes or
#    edges, however they can be added later when writing to
#    a GraphML file format[1]
# - networkx support node and edge attributes, so we can keep
#    the partition assignment with the node and also support
#    virtual nodes without needing to maintain a seperate
#    data structure.
# - the most sensible method for loading the graph data is to
#    read the METIS file with networkit, convert the graph to
#    a networkx graph, then read the METIS file once again
#    and load the node weights into a networkx node attribute
#
# Writing data
# - to be able to write the output data with the partition
#    each node is assigned to, a suitable file format to write
#    to is needed
# - writing to a METIS file will lose the partition assignments
# - if we use networkit to write the data, then the only function
#    available is GraphMLWriter()
# - networkx provides a richer set of output methods which
#    preserve the partition assignment
# - using networkit to write GML data causes a loss of edge weights and node weights
# - using networkx to write GML data preserves node and edge weights
# [1]: https://networkit.iti.kit.edu/data/uploads/docs/NetworKit-Doc/python/html/graphio.html#networkit.graphio.GraphMLWriter

# read METIS file
print("Loading graph data...")
nkG = networkit.graphio.METISGraphReader().read(DATA_FILENAME)
#U = nx.read_gml(DATA_FILENAME, label='id')
#G = U.to_directed()

# convert to networkx Graph
G = networkit.nxadapter.nk2nx(nkG)
#networkit.graphio.GMLGraphWriter().write(nkG, "data/restreaming_fennel.gml")

# add node weights from METIS file
with open(DATA_FILENAME, "r") as f:
    
    # read meta data from first line
    first_line = next(f)
    (m_nodes, m_edges, _) = first_line.split()
    m_nodes = int(m_nodes)
    m_edges = int(m_edges)

    for i, line in enumerate(f):
        if not line.strip():
            raise Exception("Error: data file contains a blank line")
        weight = line.split()[0]
        G.add_nodes_from([i], weight=str(weight))

edges = np.array(G.edges(), dtype=np.int32)
edge_weights = np.array([x[2]['weight'] for x in G.edges(data=True)], dtype=np.float32)
node_weights = np.array([x[1]['weight'] for x in G.nodes(data=True)], dtype=np.float32)

# sanity check
assert (m_nodes == G.number_of_nodes())
assert (m_nodes == len(node_weights))
assert (m_edges == G.number_of_edges())
assert (m_edges == len(edge_weights))
assert (m_edges == len(edges))

print("Nodes: {}".format(G.number_of_nodes()))
print("Edges: {}".format(G.number_of_edges()))

/home/sami/py-graph/env/lib/python3.4/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


Loading graph data...
Nodes: 1000
Edges: 2939


In [2]:
%load_ext Cython
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [3]:
%%cython
import numpy as np
from shared import fixed_width_print

cdef int UNMAPPED = -1

def fennel(int[:,::] edges,
           float[::] edge_weights,
           float[::] node_weights,
           int num_partitions,
           int[::] partition,
           int[::] fixed,
           float alpha,
           int debug):
    """
    This algorithm favors a cluster if it has many neighbors of a node, but
    penalizes the cluster if it is close to capacity.

    edges: An [:,2] array of edges.
    edge_weights: An [:,2] array of edge weights. Length should match number of edges.
    node_weights: An [:,2] array of node weights. Length should match number of nodes.
    num_partitions: How many partitions we are breaking the graph into.
    partition: A previous partition of the nodes. Set to -1's if a node has not been assigned.
    fixed: An array to denote which nodes in the partition have been locked in place.
    alpha:
    debug: Prints helpful debug information.

    Returns: A new partition.
    """

    cdef int num_nodes = len(node_weights)
    cdef float[::] partition_sizes = None

    # The output partition
    if partition is None:
        partition = np.repeat(np.int32(UNMAPPED), num_nodes)
        partition_sizes = np.zeros(num_partitions, dtype=np.float32)
    else:
        partition_sizes = np.zeros(num_partitions, dtype=np.float32)
        
        for p in range(0, len(partition)):
            if partition[p] >= 0:
                partition_sizes[partition[p]] += 1 # XXX: should use node_weights

        #partition_sizes = np.bincount(partition,
        #                              weights=node_weights,
        #                              minlength=num_partitions).astype(np.float32)

    if fixed is None:
        fixed = np.repeat(np.int32(UNMAPPED), num_nodes)

    cdef float[::] partition_votes = np.zeros(num_partitions, dtype=np.float32)

    cdef int last_left = edges[0,0]
    cdef int i = 0
    cdef int left = 0
    cdef int right = 0
    cdef int arg = 0
    cdef int max_arg = 0
    cdef float max_val = 0
    cdef float val = 0
    cdef int len_edges = len(edges)
    cdef int previous_assignment = 0

    for i in range(len_edges):
        left = edges[i,0]
        right = edges[i,1]

        if last_left != left:
            if fixed[last_left] != UNMAPPED:
                if debug:
                    print("Skipping node {}".format(last_left))
                partition_votes[:] = 0
                last_left = left

            else:
                # New left node, so we have to assign last left

                if debug:
                    print("Assigning node {}".format(last_left))
                    print("\tPn = Votes - Alpha x Size")

                # Remember placement of last_left in the previous assignment
                previous_assignment = partition[last_left]

                max_arg = 0
                max_val = partition_votes[0] - alpha * partition_sizes[0]
                if debug:
                    print("\tP{} = {} - {} x {} = {}".format(0,
                                                             partition_votes[0],
                                                             alpha,
                                                             partition_sizes[0],
                                                             max_val))

                if previous_assignment == 0:
                    # We remove the node from its current partition before
                    # deciding to re-add it, so subtract alpha to give
                    # result of 1 lower partition size.
                    max_val += alpha

                for arg in range(1, num_partitions):
                    val = partition_votes[arg] - alpha * partition_sizes[arg]

                    if debug:
                        print("\tP{} = {} - {} x {} = {}".format(arg,
                                                                 partition_votes[arg],
                                                                 alpha,
                                                                 partition_sizes[arg],
                                                                 val))
                    if previous_assignment == arg:
                        # See comment above
                        val += alpha
                    if val > max_val:
                        max_arg = arg
                        max_val = val

                if max_arg != previous_assignment:
                    partition[last_left] = max_arg
                    partition_sizes[max_arg] += node_weights[last_left]
                    if previous_assignment != UNMAPPED:
                        partition_sizes[previous_assignment] -= node_weights[last_left]

                partition_votes[:] = 0

                if debug:
                    print("\tassigned to P{}".format(partition[last_left]))
                    fixed_width_print(np.asarray(partition))
                    fixed_width_print(np.asarray(fixed))

                last_left = left

        if partition[right] != UNMAPPED:
            partition_votes[partition[right]] += edge_weights[i]

    # Clean up the last assignment
    if fixed[left] == UNMAPPED:
        if debug:
            print("Assigning last node {}".format(left))

        max_arg = 0
        max_val = 0
        for arg in range(0, num_partitions):
            val = partition_votes[arg] - alpha * partition_sizes[arg]

            if debug:
                print("\tP{} = {} - {} x {} = {}".format(arg,
                                                         partition_votes[arg],
                                                         alpha,
                                                         partition_sizes[arg],
                                                         val))

            if val > max_val:
                max_arg = arg
                max_val = val

        partition[left] = max_arg
        if debug:
            print("\tassigned to P{}".format(partition[left]))

    # Assign single nodes
    for n in range(0, len(partition)):
        if partition[n] == -1:
            partition[n] = 0
            
    return (np.asarray(partition), np.asarray(fixed))

In [4]:
%%cython
import numpy as np
import networkx as nx

cdef int UNMAPPED = -1

def get_votes(graph, int node, float[::] edge_weights, int num_partitions, int[::] partition):
    seen = set()
    cdef float[::] partition_votes = np.zeros(num_partitions, dtype=np.float32)

    # find all neighbors from whole graph
    node_neighbors = list(nx.all_neighbors(graph, node))
    node_neighbors = [x for x in node_neighbors if x not in seen and not seen.add(x)]

    # calculate votes based on neighbors placed in partitions
    for n in node_neighbors:
        if partition[n] != UNMAPPED:
            partition_votes[partition[n]] += edge_weights[n]
            
    return partition_votes

def get_assignment(int node,
                   float[::] node_weights,
                   int num_partitions,
                   int[::] partition,
                   float[::] partition_votes,
                   float alpha,
                   int debug):

    cdef int arg = 0
    cdef int max_arg = 0
    cdef float max_val = 0
    cdef float val = 0
    cdef int previous_assignment = 0

    assert partition is not None, "Blank partition passed"

    cdef float[::] partition_sizes = np.zeros(num_partitions, dtype=np.float32)
    for p in range(0, len(partition)):
        if partition[p] >= 0:
            partition_sizes[partition[p]] += 1 # XXX: should use node_weights
    
    if debug:
        print("Assigning node {}".format(node))
        print("\tPn = Votes - Alpha x Size")

    # Remember placement of node in the previous assignment
    previous_assignment = partition[node]

    max_arg = 0
    max_val = partition_votes[0] - alpha * partition_sizes[0]
    if debug:
        print("\tP{} = {} - {} x {} = {}".format(0,
                                                 partition_votes[0],
                                                 alpha,
                                                 partition_sizes[0],
                                                 max_val))

    if previous_assignment == 0:
        # We remove the node from its current partition before
        # deciding to re-add it, so subtract alpha to give
        # result of 1 lower partition size.
        max_val += alpha

    for arg in range(1, num_partitions):
        val = partition_votes[arg] - alpha * partition_sizes[arg]

        if debug:
            print("\tP{} = {} - {} x {} = {}".format(arg,
                                                     partition_votes[arg],
                                                     alpha,
                                                     partition_sizes[arg],
                                                     val))
        if previous_assignment == arg:
            # See comment above
            val += alpha
        if val > max_val:
            max_arg = arg
            max_val = val

    # XXX: partition_sizes is re-calculated at the beginning, so just return max_arg
    #if max_arg != previous_assignment:
    #    partition[node] = max_arg
    #    partition_sizes[max_arg] += node_weights[node]
    #    if previous_assignment != UNMAPPED:
    #        partition_sizes[previous_assignment] -= node_weights[node]

    if debug:
        print("\tassigned to P{}".format(max_arg)) #partition[node]))

    return max_arg

def fennel_rework(graph, 
                  float[::] edge_weights,
                  float[::] node_weights,
                  int num_partitions,
                  int[::] assignments,
                  int[::] fixed,
                  float alpha,
                  int debug):

    single_nodes = []
    for n in range(0, graph.number_of_nodes()):

        # Exclude single nodes, deal with these later
        neighbors = list(nx.all_neighbors(graph, n))
        if not neighbors:
            single_nodes.append(n)
            continue
            
        # Skip fixed nodes
        if fixed[n] != UNMAPPED:
            if debug:
                print("Skipping node {}".format(n))
            continue

        partition_votes = get_votes(graph, n, edge_weights, num_partitions, assignments)
        assignments[n] = get_assignment(n, node_weights, num_partitions, assignments, partition_votes, alpha, debug)

    # Assign single nodes
    for n in single_nodes:
        if assignments[n] == UNMAPPED:
            assignments[n] = 0

    return np.asarray(assignments)

In [5]:
# Number of shelters
num_partitions = 4

# Order of people arriving
arrivals = list(range(0, G.number_of_nodes()))
#random.shuffle(arrivals)

# The number of iterations when making prediction model
num_iterations = 10

# Alpha value used in prediction model
prediction_model_alpha = 0.011756

# Percentage of prediction model to use before discarding
# When set to 0, prediction model is discarded, useful for one-shot
prediction_model_cut_off = 0.10

# Alpha value used in one-shot (when restream_batches set to 1)
one_shot_alpha = 0.5

# Number of arrivals to batch before recalculating alpha and restreaming.
# When set to 1, one-shot is used with alpha value from above
restream_batches = 10

In [6]:
import shared
UNMAPPED = -1

# reset
assignments = np.repeat(np.int32(UNMAPPED), len(node_weights))
fixed = np.repeat(np.int32(UNMAPPED), len(node_weights))

print("PREDICTION MODEL")
print("----------------\n")
print("WASTE\t\tCUT RATIO\tMISMATCH")
for i in range(num_iterations):
    alpha = prediction_model_alpha
    #assignments, fixed = fennel(edges, edge_weights, node_weights, num_partitions, assignments, fixed, alpha, 0)
    assignments = fennel_rework(G, edge_weights, node_weights, num_partitions, assignments, fixed, alpha, 0)

    x = shared.score(assignments, edges)
    print("{0:.5f}\t\t{1:.10f}\t{2}".format(x[0], x[1], x[2]))

print("\nAssignments:")
fixed_width_print(assignments)

nodes_fixed = len([o for o in fixed if o == 1])
print("\nFixed: {}".format(nodes_fixed))

shared.print_partitions(assignments, num_partitions, node_weights)

PREDICTION MODEL
----------------

WASTE		CUT RATIO	MISMATCH
0.03200		0.2177611432	640
0.04000		0.1422252467	418
0.00400		0.1286151752	378
0.00400		0.1269139163	373
0.00400		0.1255529092	369
0.00000		0.1241919020	365
0.00400		0.1201088806	353
0.00000		0.1184076216	348
0.00400		0.1180673699	347
0.00400		0.1160258591	341

Assignments:
[ 0  1  2  0  1  0  1  3  0  0  2  0  0  1  0  2  3  2  2  0  0  3  2  3  0  1  3  1  2  0  0  2  0  1  2  3  0  3  2  0  2  0  2  1  0  3  1  3  3  2  0  1  2  0  0  2  3  0  1  0  2  1  0  1  1  1  1  2  3  2  1  0  0  1  1  3  1  1  0  1  2  3  1  0  1  2  0  2  3  0  1  3  3  0  1  2  3  0  0  1  1  2  3  1  1  0  1  0  2  0  2  1  2  2  3  1  3  1  0  2  1  0  0  3  1  1  3  2  2  1  0  0  1  1  0  3  1  2  3  1  1  2  3  2  3  2  1  2  0  0  3  1  1  2  1  2  1  3  3  0  1  3  0  0  0  2  3  2  3  0  0  1  0  1  2  1  0  2  1  1  2  0  0  0  1  0  2  1  1  2  3  2  2  0  0  3  3  2  3  0  2  1  0  3  2  0  2  1  2  2  1  0  3  0  3  0  0  1  2  3  0  

In [7]:
cut_off_value = int(prediction_model_cut_off * G.number_of_nodes())
if prediction_model_cut_off == 0:
    print("Discarding prediction model\n")
else:
    print("Assign first {} arrivals using prediction model, then discard\n".format(cut_off_value))

# fix arrivals
for a in arrivals:
    nodes_fixed = len([o for o in fixed if o == 1])
    if nodes_fixed >= cut_off_value:
        break
    fixed[a] = 1

# remove nodes not fixed, ie. discard prediction model
for i in range(0, len(assignments)):
    if fixed[i] == -1:
        assignments[i] = -1

print("WASTE\t\tCUT RATIO\tMISMATCH")
x = shared.score(assignments, edges, num_partitions)
print("{0:.5f}\t\t{1:.10f}\t{2}".format(x[0], x[1], x[2]))

print("\nAssignments:")
fixed_width_print(assignments)

nodes_fixed = len([o for o in fixed if o == 1])
print("\nFixed: {}".format(nodes_fixed))

shared.print_partitions(assignments, num_partitions, node_weights)

Assign first 100 arrivals using prediction model, then discard

WASTE		CUT RATIO	MISMATCH
0.03200		0.1861177271	547

Assignments:
[ 0  1  2  0  1  0  1  3  0  0  2  0  0  1  0  2  3  2  2  0  0  3  2  3  0  1  3  1  2  0  0  2  0  1  2  3  0  3  2  0  2  0  2  1  0  3  1  3  3  2  0  1  2  0  0  2  3  0  1  0  2  1  0  1  1  1  1  2  3  2  1  0  0  1  1  3  1  1  0  1  2  3  1  0  1  2  0  2  3  0  1  3  3  0  1  2  3  0  0  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1

In [8]:
if restream_batches == 1:
    print("One-shot assignment mode")
    print("------------------------\n")
else:
    print("Re-streaming in batches of {}".format(restream_batches))
    print("--------------------------------\n")

batch_arrived = []
print("WASTE\t\tCUT RATIO\tMISMATCH\tALPHA")
for a in arrivals:
    # check if node is already arrived
    if fixed[a] == 1:
        continue

    # one-shot assigment: assign each node as it arrives
    if restream_batches == 1:
        alpha = one_shot_alpha
        partition_votes = get_votes(G, a, edge_weights, num_partitions, assignments)
        assignments[a] = get_assignment(a, node_weights, num_partitions, assignments, partition_votes, alpha, 0)
        fixed[a] = 1
        
        x = shared.score(assignments, edges, num_partitions)
        print("{0:.5f}\t\t{1:.10f}\t{2}\t\t{3:.10f}".format(x[0], x[1], x[2], alpha))
        continue
        
    batch_arrived.append(a)

    if restream_batches == len(batch_arrived):

        # make a subgraph of all arrived nodes
        nodes_arrived = []
        for n in range(0, len(assignments)):
            if fixed[n] == 1 or n in batch_arrived:
                nodes_arrived.append(n)
        Gsub = G.subgraph(nodes_arrived)

        # recalculate alpha
        # XXX: as it's an undirected graph, edges_arrived is actually double, divide by 2?
        nodes_fixed = len([o for o in fixed if o == 1])
        edges_arrived = Gsub.number_of_edges()
        alpha = (edges_arrived) * (num_partitions / (nodes_fixed + len(batch_arrived))**2)

        # restream
        for n in batch_arrived:
            partition_votes = get_votes(Gsub, n, edge_weights, num_partitions, assignments)
            assignments[n] = get_assignment(n, node_weights, num_partitions, assignments, partition_votes, alpha, 0)
            fixed[n] = 1

        x = shared.score(assignments, edges, num_partitions)
        print("{0:.5f}\t\t{1:.10f}\t{2}\t\t{3:.10f}".format(x[0], x[1], x[2], alpha))
        batch_arrived = []

# remove nodes not fixed
for i in range(0, len(assignments)):
    if fixed[i] == -1:
        assignments[i] = -1

print("\nAssignments:")
fixed_width_print(assignments)

nodes_fixed = len([o for o in fixed if o == 1])
print("\nFixed: {}".format(nodes_fixed))

shared.print_partitions(assignments, num_partitions, node_weights)

Re-streaming in batches of 10
--------------------------------

WASTE		CUT RATIO	MISMATCH	ALPHA
0.02600		0.2004083021	589		0.0102479339
0.02000		0.2153793807	633		0.0105555556
0.01800		0.2300102076	676		0.0101775148
0.01200		0.2398775094	705		0.0104081633
0.00200		0.2500850629	735		0.0117333333
0.00400		0.2664171487	783		0.0120312500
0.00200		0.2769649541	814		0.0119031142
0.00400		0.2895542702	851		0.0123456790
0.00600		0.3045253488	895		0.0126315789
0.00400		0.3171146649	932		0.0122000000
0.00200		0.3273222184	962		0.0125170068
0.00000		0.3416127935	1004		0.0126446281
0.00600		0.3555631167	1045		0.0121739130
0.00400		0.3654304185	1074		0.0118750000
0.00200		0.3810820007	1120		0.0115200000
0.00400		0.3916298061	1151		0.0114201183
0.00200		0.4038788704	1187		0.0111934156
0.00400		0.4066008847	1195		0.0113775510
0.00600		0.4161279347	1223		0.0115576694
0.00400		0.4273562436	1256		0.0114222222
0.01000		0.4379040490	1287		0.0114464100
0.00400		0.4470908472	1314		0.0118359375
0.01000		0.45

In [9]:
# Add partition attribute to nodes and write to file
for i in range(0, len(assignments)):
    G.add_nodes_from([i], partition=str(assignments[i]))
nx.write_gml(G, "test.gml")